# Using scanpy for scRNA-seq analysis

In this notebook we will be using scanpy.

The notebook also includes some code to run pseudotime analysis, but this will be explored on a different tutorial in a later session.

We start by installing some required packages

In [ ]:
!pip install scanpy scvelo anndata umap-learn matplotlib leidenalg

And we import those packages

In [ ]:
import scanpy as sc # package for manipulating single cell data
sc.settings.verbosity = 3 # getting more info from scanpy
import scvelo as scv # package based on scanpy to do advanced modelling of RNA velocity

import pandas as pd # general data wranggling
import numpy as np # numeric data wranggling

# setting the seed for RNG
import random
random.seed(10)

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
## magic to allow in-line figures
%matplotlib inline

Setting visualisation parameters for scvelo

In [ ]:
scv.settings.verbosity = 3  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.presenter_view = True  # set max width size for presenter view
scv.settings.set_figure_params('scvelo')  # for beautified visualization

## Basic concepts about AnnData objects

Obtain the data - we're using one of the datasets included in scVelo. This is a mouse pancreas development dataset from https://journals.biologists.com/dev/article/146/12/dev173849/19483/Comprehensive-single-cell-mRNA-profiling-reveals-a

In [ ]:
adata = scv.datasets.pancreas()
adata

In [ ]:
adata.obs.head()

You can just see the number of rows and columns of an AnnData object (or any DataFrame) with `.shape`.

In [ ]:
adata.shape

`.obs` contains the cell metadata (i.e. annotations, etc.).

In [ ]:
adata.obs.head(n = 10)

`.var` contains the gene metadata

In [ ]:
adata.var.head(n = 10)

Each of these tables is implemented as a Pandas `DataFrame`. The row names are in `.index`, and the column names are in `.columns`. These, in turn, are numpy arrays, that can be accessed with `.values`

In [ ]:
adata.var.index.values

In [ ]:
adata.obs.columns.values

You can count the number of elements in each category with `.value_counts()`

In [ ]:
adata.obs["clusters_coarse"].value_counts()

And you can cross tabulate two columns using `.crosstab()` (a function in the pandas package)

In [ ]:
pd.crosstab(adata.obs["clusters_coarse"], adata.obs["clusters"])

Doubts about how to use a function?

In [ ]:
??sc.tl.pca

## Basic analysis

Adapted from: https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html

Check the expression of the most expressed genes

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20)

Filtering cells and genes based on their expression level. By convention, cells are not considered as such with less than 100 different expressed genes (and usually we filter above that). Genes should also be present in a non-trivial portion of the data. We have 0.1% (or 3/~3000 cells) here.

In [ ]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
adata

Now we're calculating some important QC metrics.

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
adata.obs.head(10)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

And we can now filter the data. We'll be removing some cells that have too many counts or MT% (even though with the latter I'm just being picky)

In [ ]:
adata = adata[adata.obs.total_counts < 16000, :]
adata = adata[adata.obs.pct_counts_mt < 2, :]

Now we can normalise and log-transform the data

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

And then calculate the highly-variable genes

In [ ]:
sc.pp.highly_variable_genes(adata, min_mean=0.01, max_mean=3, min_disp=0.5)

We can use scanpy's plotting functions to examine gene variability and expression level

In [ ]:
sc.pl.highly_variable_genes(adata)

scanpy doesn't automatically keep different slots for the unchanged data, even after filtering to use only HVG. So first we must save the normalised data into a "raw" slot, so we can later perform differential expression

In [ ]:
adata.raw = adata

In [ ]:
adata = adata[:, adata.var.highly_variable]

And now we filter the main matrix to only contain the HVG

In [ ]:
sc.pp.regress_out(adata, ['total_counts'])

We scale those genes

In [ ]:
sc.pp.scale(adata)

And run a PCA

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')

In [ ]:
sc.pl.pca(adata, color=['Por', 'Col18a1'])

In [ ]:
adata

How much variance does each PC explain?

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True)

We can now calculate the neighbourhood graph for the cells in PC space

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=25)

And get a UMAP

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color=['Col18a1'], use_raw=False)

We also cluster the data

In [ ]:
sc.tl.leiden(adata, resolution = 0.4)

In [ ]:
sc.pl.umap(adata, color=['leiden'])

Now we can calculate marker genes for all these clusters! We will also be plotting the top 25 genes for each of them.

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon', use_raw = True)
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

We can also print the top 5 genes for all clusters as a table

In [ ]:
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(5)

And we can include more information by also showing the p-value for the DE test

In [ ]:
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals']}).head(5)

How does cluster 0 compare against the rest for its top 8 marker genes?

In [ ]:
sc.pl.rank_genes_groups_violin(adata, groups='0', n_genes=8)

We can also do 1 vs 1 comparisons

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', groups=['0'], reference='1', method='wilcoxon')
sc.pl.rank_genes_groups(adata, groups=['0'], n_genes=20)
sc.pl.rank_genes_groups_violin(adata, groups='0', n_genes=8)

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', groups=['1'], reference='0', method='wilcoxon')
sc.pl.rank_genes_groups(adata, groups=['1'], n_genes=20)
sc.pl.rank_genes_groups_violin(adata, groups='1', n_genes=8)

Lets define a set of marker genes here for further plotting

In [ ]:
marker_genes = ['Dlk1', 'Spp1', 'Krt7', 'Top2a', 'Vtn', 'Sox9', 'Isl1', 'Cldn4', 'Pyy']

In [ ]:
sc.pl.violin(adata, marker_genes, groupby='leiden')

In [ ]:
sc.pl.umap(adata, color=['leiden']+marker_genes, legend_loc='on data', title='', frameon=False, save='.pdf')

To get an overview of cell populations, we can look at heatmaps/dotplots/violin matrices

In [ ]:
sc.pl.dotplot(adata, marker_genes, groupby='clusters');

In [ ]:
sc.pl.stacked_violin(adata, marker_genes, groupby='clusters');

And this is how you save the data (but you don't have to do it here)

In [ ]:
# adata.write("path/to/output/adata.h5ad")

## Pseudotime analysis

### PAGA - Partition-based graph abstraction (see https://scanpy-tutorials.readthedocs.io/en/latest/paga-paul15.html)

PAGA can help us understand the relationship between clusters, and lead to a cluster-based pseudotime ordering of the data

In [ ]:
sc.tl.paga(adata, groups='leiden')

In [ ]:
sc.pl.paga(adata, color=['leiden']+marker_genes)

And now we can get a new graph-based projection for the data

In [ ]:
sc.tl.draw_graph(adata, init_pos='paga')

In [ ]:
sc.pl.draw_graph(adata, color=['leiden'] + marker_genes, legend_loc='on data')

### Diffusion pseudotime

To run a diffusion-based pseudotime algorithm, we have first to define a root cell

In [ ]:
adata.uns['iroot'] = np.flatnonzero(adata.obs['leiden']=='16')[0]

Now we get the diffusion pseudotime

In [ ]:
sc.tl.dpt(adata)

And plot it

In [ ]:
sc.pl.draw_graph(adata, color=['leiden', 'dpt_pseudotime'], legend_loc='on data')

In [ ]:
sc.pl.paga(adata, color=['leiden'])

And we can plot change in gene expression along pseudotime

In [ ]:
scv.pl.scatter(adata, x='dpt_pseudotime', y=marker_genes[0], frameon=False, use_raw = None)

The follwing code can be found in the tutorial https://scanpy-tutorials.readthedocs.io/en/latest/paga-paul15.html. Should show plotting for each defined branch, but unfortunately it doesn't work

In [ ]:
paths = [('beta', [17, 12, 7, 13, 10,0,4,9,5,3,2,6,11]),
         ('alpha', [17, 12, 7, 13, 10,0,4,9,5,3,2,16,1]),
         ('delta', [17, 12, 7, 13, 10,0,4,9,5,3,2,15,8])]

_, axs = plt.subplots(ncols=3, figsize=(6, 2.5), gridspec_kw={'wspace': 0.05, 'left': 0.12})
plt.subplots_adjust(left=0.05, right=0.98, top=0.82, bottom=0.2)

for ipath, (descr, path) in enumerate(paths):
  _, data = sc.pl.paga_path(
      adata, path, marker_genes,
      show_node_names=False,
      ax=axs[ipath],
      ytick_fontsize=12,
      left_margin=0.15,
      n_avg=50,
      annotations=['dpt_pseudotime'],
      #show_yticks=True if ipath==0 else False,
      show_colorbar=True,
      color_map='Greys',
      groups_key='leiden',
      color_maps_annotations={'dpt_pseudotime': 'viridis'},
      title='{} path'.format(descr),
      return_data=True,
      show=False)

plt.show()

## scVelo analysis for transcriptional dynamics

The dataset we're using already includes a quantification for spliced and unspliced transcripts

In [ ]:
scv.utils.show_proportions(adata)

Further, we need the first and second order moments (basically mean and uncentered variance) computed among nearest neighbors in PCA space. First order is needed for deterministic velocity estimation, while stochastic estimation also requires second order moments.

In [ ]:
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)

The gene-specific velocities are obtained by fitting a ratio between precursor (unspliced) and mature (spliced) mRNA abundances that well explains the steady states (constant transcriptional state) and then computing how the observed abundances deviate from what is expected in steady state



In [ ]:
scv.tl.velocity(adata)

Now we compute a velocity graph for low-dimensional embedding



In [ ]:
scv.tl.velocity_graph(adata)

In [ ]:
sc.pl.umap(adata, color=['leiden'])

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap',  color = "leiden")

In [ ]:
scv.pl.velocity_embedding(adata, basis='umap', arrow_length=2, arrow_size=1.5, dpi=150)

Dynamic modelling is done to get the full transcriptional dynamics (see https://scvelo.readthedocs.io/DynamicalModeling/). We can get transcription, splicing and degradation rates (not covered here)


In [ ]:
scv.tl.recover_dynamics(adata)

In [ ]:
scv.tl.velocity(adata, mode='dynamical')
scv.tl.velocity_graph(adata)

We can also obtain a latent pseudotime estimate

In [ ]:
scv.tl.latent_time(adata)
scv.pl.scatter(adata, color='latent_time', color_map='gnuplot', size=80, colorbar=True)

In [ ]:
top_genes = adata.var['fit_likelihood'].sort_values(ascending=False).index[:300]
scv.pl.heatmap(adata, var_names=top_genes, tkey='latent_time', n_convolve=100, col_color='clusters')

In [ ]:
scv.pl.scatter(adata, x='latent_time', y=marker_genes[0], frameon=False)

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap')